In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json_lines

In [2]:
path = "../data/raw"
# claim_response= pd.read_json(f"{path}/ClaimResponse.ndjson", lines=True)


In [3]:
claim_resp = pd.read_json(f"{path}/ClaimResponse.ndjson", lines=True, nrows=1)

In [4]:
claim_resp

,contained,created,extension,id,identifier,insurer,meta,outcome,patient,request,resourceType,status,type,use
0,"[{'birthDate': '1944-05-25', 'extension': [{'u...",2025-09-03T22:20:31+00:00,[{'url': 'https://bluebutton.cms.gov/resources...,f-LTEwMDAwMDAzNTUxNzU5,[{'system': 'https://bluebutton.cms.gov/resour...,{'identifier': {'value': 'CMS'}},{'lastUpdated': '2023-05-11T21:17:37.364+00:00'},queued,{'reference': '#patient'},{'reference': 'Claim/f-LTEwMDAwMDAzNTUxNzU5'},ClaimResponse,active,"{'coding': [{'code': 'institutional', 'display...",claim


In [6]:
for key, value in claim_resp.items():
    print(f"key: {key}")
    for item in value:
            if isinstance(item, dict):
                for key, value in value.items():
                    print(f"\tkey: ({key}) value ({value})\n")
            #print(item)
            elif isinstance(item, list):
                for i in item:
                    if isinstance(i, dict):
                        for key, value in i.items():
                            if isinstance(value, list):
                                for y in value:
                                    print(f"\t\tkey: {key} list: ({y})\n")
                            else:
                                print(f"\t\tkey: ({key}) value ({value})\n")
                    else:
                        print(f"{i}\n")
            else:
                print(f"\tvalue: {item}\n")


key: contained
		key: (birthDate) value (1944-05-25)

		key: extension list: ({'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-sex', 'valueCode': '248152002'})

		key: (gender) value (female)

		key: (id) value (patient)

		key: identifier list: ({'system': 'http://hl7.org/fhir/sid/us-mbi', 'type': {'coding': [{'code': 'MC', 'display': "Patient's Medicare Number", 'system': 'http://terminology.hl7.org/CodeSystem/v2-0203'}]}, 'value': '1S00E00JK17'})

		key: name list: ({'family': 'Wiza601', 'given': ['Patrina117'], 'text': 'Patrina117 Wiza601 ([max 10 chars of first], [max 15 chars of last])'})

		key: (resourceType) value (Patient)

key: created
	value: 2025-09-03T22:20:31+00:00

key: extension
		key: (url) value (https://bluebutton.cms.gov/resources/variables/fiss/curr-status)

		key: (valueCoding) value ({'code': 'A', 'system': 'https://bluebutton.cms.gov/resources/variables/fiss/curr-status'})

		key: (url) value (https://bluebutton.cms.gov/resources/variables/fiss/